# How to Train a YOLO26 Object Detection Model with Custom Data

We will show you how to train a YOLO26 detection model with your images and annotations and export to a Core ML model which can be used for auto labeling on RectLabel.

### Use GPU

Let's make sure that we have access to GPU. We can use `nvidia-smi` command to do that. In case of any problems navigate to `Runtime` -> `Change runtime type` -> `Hardware accelerator`, set it to `GPU`, and then click `Save`.

In [1]:
!nvidia-smi

Sun Feb 22 21:47:44 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8              8W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Install YOLO26

In [2]:
%pip install -q ultralytics

### Download training images and annotations

Download training images and annotations. You can use these or replace them with your own data.

In [7]:
!mkdir datasets
%cd datasets
!wget -q https://huggingface.co/datasets/rectlabel/datasets/resolve/main/converse_vans_detection.zip
!unzip -q converse_vans_detection.zip
%cd ..

/content/datasets
/content


## Fine-tune YOLO26 on custom dataset

Create a workspace folder and start training from the workspace folder. Make sure the datasets path in the yaml file.

In [8]:
!mkdir workspace
%cd workspace
!mv ../datasets/converse_vans_detection/converse_vans_detection.yaml .

/content/workspace


In [9]:
!yolo task=detect mode=train model=yolo26n.pt data=converse_vans_detection.yaml epochs=100 imgsz=640 plots=True

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=converse_vans_detection.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, in

Here are trained models.

In [10]:
!ls runs/detect/train/weights

best.pt  last.pt


Move the best model to the current folder and export to a Core ML model.

In [11]:
!mv runs/detect/train/weights/best.pt .

In [12]:
!yolo export model=best.pt format=coreml

Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.10.0+cu128 CPU (Intel Xeon CPU @ 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO26n summary (fused): 122 layers, 2,375,226 parameters, 0 gradients, 5.2 GFLOPs

PyTorch: starting from 'best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 300, 6) (5.1 MB)
requirements: Ultralytics requirement ['coremltools>=9.0'] not found, attempting AutoUpdate...
Using Python 3.12.12 environment at: /usr
Resolved 12 packages in 260ms
Prepared 3 packages in 139ms
Installed 3 packages in 15ms
 + cattrs==26.1.0
 + coremltools==9.0
 + pyaml==26.2.1

requirements: AutoUpdate success ✅ 0.8s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect

scikit-learn version 1.6.1 is not supported. Minimum required version: 0.17. Maximum required version: 1.5.1. Disabling scikit-learn conversion API.
XGBoost version 3.2

Zip the Core ML model and download it from the File browser at the left hand. You can auto label images using the Core ML model on RectLabel.

In [13]:
!zip -r best.mlpackage.zip best.mlpackage

  adding: best.mlpackage/ (stored 0%)
  adding: best.mlpackage/Manifest.json (deflated 59%)
  adding: best.mlpackage/Data/ (stored 0%)
  adding: best.mlpackage/Data/com.apple.CoreML/ (stored 0%)
  adding: best.mlpackage/Data/com.apple.CoreML/weights/ (stored 0%)
  adding: best.mlpackage/Data/com.apple.CoreML/weights/weight.bin (deflated 8%)
  adding: best.mlpackage/Data/com.apple.CoreML/model.mlmodel (deflated 88%)
